# XGBoost: A Scalable Tree Boosting System

## Tianqi Chen and Carlos Guestrin
## University of Washington 2016


### Гавриленко Арсений, МФТИ 2019 

<img src="pictures/meme1.png" alt="drawing" width="600"/>

## Содержание статьи

### I Новый алгортим для работы с разреженными данными
### II Алгоритм квантильного разбиения взвешенных данных
### III Мультипоточность и оптимизация работы с данными


 ## <font size=5 color=black> <center> 1  Введение </center></font> 


<h2><center>Популярность</center></h2>
<img src="pictures/Popularity.jpg" alt="drawing" width="800"/>

<h2><center> Эффективность </center></h2>
<img src="pictures/speed.png" alt="drawing" width="600"/>


 ## <font size=5 color=black> <center> 2.1 Формализуем модель обучения </center></font> 


### Пусть данные состоят из n объектов и m признаков - $\mathcal{D} = \{(x_i, y_i)\}$ ($|\mathcal{D}| = n, x_i\in \mathbb{R}^m, y_i \in \mathbb{R}$) и пусть наш ансамбль алгоритмов состоит из K деревьев, тогда наше предсказание для каждого объекта :

### <center> $\hat{y}_i = \phi(x_i) = \sum^K_{k=1} f_k(x_i), \ \ f_k\in \mathcal{F}$</center>

### $\mathcal{F}=\{f(x) = w_{q(x)}\} ( q : \mathbb{R}^m \rightarrow T, w\in \mathbb{R}^T) $ То есть $\mathcal{F}$ - пространство регрессионных деревьев, T - количество листьев в дереве

<img src="pictures/tree_model.png" alt="drawing" width="700"/>

### В процессе обучения будем минимизировать регуляризованную функцию потерь: 


### <center>$ \mathcal{L}(\phi) = \sum_{i} l( \hat{y}_i, y_i ) + \sum_{k}\Omega( f_{k} ) \ \ \ \ (2)$   </center>
### где $ \Omega(f) = \gamma T + \frac{1}{2} \lambda \|w\|^2$ и $l( \hat{y}_i, y_i )$ - выпуклая дифференцируемая функция потерь

<img src="pictures/newstrip.png" alt="drawing" width="1400"/>

## <font size=5 color=black> <center> 2.1.0. Решающие деревья</center> </font> 


<img src="pictures/trees.png" alt="drawing" width="700"/>

### $[x^j \leq t]$ j-ый признак, t - порог
### <center>$Q(X_m, j, t) = \frac{|X_l|}{|X_m|}\cdot H(X_l) + \frac{|X_r|}{|X_m|} \cdot H(X_r) \rightarrow min$</center>
### Регрессия : $H(X_m) = \frac{1}{|X_m|} \cdot  \sum_{i}^{m} ( y_i - \hat{y}(X_m))^2$
### Класификация (кр. Джини) : $H(X_m) = \sum_{k}^{m} p_k(1-p_k)$, где $p_k = \frac{1}{|X|} \cdot  \sum_{i \in X}^{K} [y_i = k]$

<img src="pictures/newstrip.png" alt="drawing" width="1400"/>


## <font size=5 color=black><center> 2.2 Градиентный бустинг на деревьях</center></font> 


<img src="pictures/golf.png" alt="drawing" width="700"/>

### Пусть $\hat{y_i}^{(t)}$ предсказание для i-ого объекта на t-ой итерации, тогда добавим $f_t$ такое, чтобы минимизировать:
### <center>$ \mathcal{L}^{(t)} = \sum_{i=1}^n l(y_i,\hat{y_i}^{(t-1)}+f_t(x_i))+\Omega(f_t)$</center>

### Воспользуемся приближением второго порядка:
### $f(a+a_1,b+b_1)=f(a,b)+f_x(a,b) a_1+f_y(a,b)\cdot b_1+\frac{1}{2}(a^{2}_{1}\cdot f_{xx}+a_1 \cdot b_1\cdot f_{xy}+a_1\cdot b_1\cdot f_{yx}+b_{1}^{2}\cdot f_{yy})$
### Так как $y_i$ у нас не меняется(метка класса) , то
### $f(a,b+b_1)=f(a,b)+f_y(a,b)\cdot b_1+\frac{1}{2}\cdot b_{1}^{2}\cdot f_{yy}$ Тогда
### <center>$\mathcal{L}^{(t)} \simeq \sum_{i=1}^n [l(y_i,\hat{y}^{(t-1)}) + g_i f_t(x_i)+\frac{1}{2}h_i f_t^2(x_i)] + \Omega(f_t)$</center>

### где  $g_i = \partial_{\hat{y}^{(t-1)}}l(y_i,\hat{y}^{(t-1)})$ и  $h_i = \partial^2_{\hat{y}^{(t-1)}}l(y_i,\hat{y}^{(t-1)})$

### $l(y_i,\hat{y}^{(t-1)})$ положительная константа, поэтому можем ее не учитывать при минимизации функции 

### <center> $ \tilde{\mathcal{L}}^{(t)} = \sum_{i=1}^n [g_i f_t(x_i)+\frac{1}{2}h_i f_t^2(x_i)] + \Omega(f_t)  \ \ \ \ (3)$ </center>

### Обозначим $I_j = \{i | q(x_i) = j\}$ - набор объектов, которые попали в j-ый лист и распишем регуляризацию на компоненты:

### $$\begin{equation}
\begin{split}
\tilde{\mathcal{L}}^{(t)}
         &=\sum^n_{i=1} [g_i f_t(x_i)+\frac{1}{2}h_if_t^2(x_i)]+\gamma T + \frac{1}{2}\lambda\sum^T_{j=1}w_j^2\\
         &=\sum^T_{j=1}[(\sum_{i\in I_j} g_i)w_j+\frac{1}{2}(\sum_{i\in I_j} h_i+\lambda)w_j^2]+\gamma T
\end{split}
\end{equation} \ \ \ \ (4)$$

### Для каждой фиксированной структуры дерева $q(x)$ можем посчитать оптимальный вес $w^*_j$ для листа $j$
### $$\begin{equation}\label{eq:leafscore}
w^*_j =-\frac{\sum_{i\in I_j} g_i}{\sum_{i\in I_j} h_i+\lambda},
\end{equation} \ \ \ \ (5)$$
### Тогда подставим и получим:
### $$\begin{equation}\label{eq:score}
\tilde{\mathcal{L}}^{(t)}(q) = - \frac{1}{2} \sum^T_{j=1}\frac{(\sum_{i\in I_j} g_i)^2}{\sum_{i\in I_j} h_i + \lambda}+\gamma T.
\end{equation} \ \ \ \ (6)
$$

### Полученую новую метрику можно использовать для нахождения качества построения структуры деревa $q(x)$

<img src="pictures/struct_score.png" alt="drawing" width="700"/>

### Невозможно рассмотреть все деревья, поэтому используют жадный алгоритм с функцией потерь: 
### если $I= I_L\cup I_R$, тогда 
### $$\begin{equation}\label{eq:gain}
      \mathcal{L}_{split} =\frac{1}{2} \left[\frac{(\sum_{i\in I_L} g_i)^2}{\sum_{i\in I_L} h_i + \lambda}+\frac{(\sum_{i\in I_R} g_i)^2}{\sum_{i\in I_R} h_i + \lambda} - \frac{(\sum_{i\in I} g_i)^2}{\sum_{i\in I} h_i + \lambda}\right] - \gamma
\end{equation}\  \  \ \ (7)$$ 

<img src="pictures/meme2.png" alt="drawing" width="700"/>

## <font size=5 color=black> <center> 2.3 Обучение на подвыборках</center> </font> 

### I Обучение не на всех признаках
### II Обучение не на всех объектах
### III Уменьшения новых весов - будем делить на  $\lambda$, чтобы уменьшить влияние новых деревьев - работает по аналогии с градиентым спуском

<img src="pictures/newstrip.png" alt="drawing" width="1400"/>

## <font size=5 color=black> <center> 3 Алгоритмы разбиения </center> </font> 

## <font size=5 color=black> <center> 3.1 Базовый точный жадный алгоритм </center> </font> 

## <font size=5 color=black> <center> 3.2 Приближенный алгоритм  </center> </font> 
### 1.  Глобальный
### 2. Локальный

<img src="pictures/1-1.png" alt="drawing" width="700"/>

## <font size=5 color=black> <center> 3.3 Стандартное квантильное разбиение </center> </font> 

### Пусть у нас есть набор(multi-set) данных состоящий из значений k-ых признаков и вторых производных $h_i$ :
$D_k=\{(x_{1k}, h_1), (x_{2k}, h_2) \cdots (x_{nk}, h_n)\}$
### Зададим такую функцию:
### $$\begin{equation}
    r_{k}(z) =\frac{1}{\sum_{(x, h)\in D_k} h} \sum_{(x, h)\in D_k, x < z} h, \ \ \ \ (8)
\end{equation}$$
### Которая равна доле объектов, у которых k-ый признак меньше порога z
### Наша цель - найти такие значения $\{s_{k1}, s_{k2}, \cdots s_{kl}\}$, чтобы потом по ним разбить, причем мы хотим разбить данные равномерно, поэтому будем использовать персентили:
### $$\begin{equation}
    |r_{k}(s_{k,j})  - r_{k}(s_{k,j+1})| < \epsilon, \ \ s_{k1} = \min_i x_{ik},  s_{kl} = \max_i x_{ik}.
\end{equation} \label{eq:quantile}$$

### Разумный вопрос - почему мы считаем h весом экземпляра?

### Вынесем из (3) $h_i$ и постоянные величины:
### $$ \tilde{\mathcal{L}}^{(t)} = \sum_{i=1}^n [g_i f_t(x_i)+\frac{1}{2}h_i f_t^2(x_i)] + \Omega(f_t) = \sum_{i=1}^n \frac{1}{2} h_i (f_t(x_i) - (-\frac{g_i}{h_i} ))^2 + \Omega(f_t) + constant $$
### $$
\sum_{i=1}^n \frac{1}{2} h_i (f_t(x_i) - (-\frac{g_i}{h_i} ))^2 + \Omega(f_t) + constant = \sum_{i=1}^n \frac{1}{2} h_i (f_t^2(x_i) + 2\cdot f_t(x_i) \frac{g_i}{h_i}+\frac{g_i^2}{h_i^2}) + \Omega(f_t) + constant = \sum_{i=1}^n (\frac{1}{2} h_i \cdot f_t^2(x_i) + f_t(x_i) \cdot g_i) + \Omega(f_t) + constant
$$
### Лосс функция для данных с меткой $- \frac{g_i}{h_i}$ и весом  $h_i$

<img src="pictures/newstrip.png" alt="drawing" width="1400"/>

## <font size=5 color=black> <center> 3.4 Взвешенное квантильное разбиение </center> </font> 

<img src="pictures/math.jpg" alt="drawing" width="400"/>

### Пусть у нас есть выборка $D_k=\{(x_{1k}, h_1), (x_{2k}, h_2) \cdots (x_{nk}, h_n)\}$, введем функцию ранг $$r_{k}(z) =\frac{1}{\sum_{(x, h)\in D_k} h} \sum_{(x, h)\in D_k, x < z} h$$



### Наша цель найти такие значения 
### $\{s_{k1}, s_{k2}, \cdots s_{kl}\}$, что
### \begin{equation}
    |r_{k}(s_{k,j})  - r_{k}(s_{k,j+1})| < \epsilon, \ \ \ \ \  s_{k1} = \min_i x_{ik}, \ \ \ \  s_{kl} = \max_i x_{ik}.
\end{equation} 


### Построение такого разбиения:

### Алгоритм состоит из двух основных компонент:
### 1. Операция слияния которая сливает два разбиения с ошибками $\epsilon_1$ и $\epsilon_2$ вместе и выдает разбиение с ошибкой  $\max(\epsilon_1,\epsilon)$ 
### 2. Операция разбиения, которая уменьшает количество элементов в блоке до b+1, и изменяет ошибку с $\epsilon$ до $\epsilon + \frac{1}{b}$.


### Датасет - $D=\{(x_1,w_1), (x_2,w_2) \cdots (x_n,w_n)\}$, где $w_i\in [0; \inf), x_i \in X$
### Пусть у нас введено отношение порядка на X
### Тогда определим функции $r^-_{D}, r^+_{D}, w_{D}, w: X \rightarrow [0; \inf)$
### $$\begin{equation}
    r^-_{D}(y) = \sum_{(x,w)\in D, x < y} w
\end{equation}$$
### $$\begin{equation}
    r^+_{D}(y) = \sum_{(x,w)\in D, x \leq y} w
\end{equation}$$
### $$\begin{equation}
    w_{D}(y) =  r^+_{D}(y)  - r^-_{D}(y) =  \sum_{(x,w)\in D, x = y} w .
\end{equation}$$

### $$\begin{equation}
    w(D) = \sum_{(x,w)\in D} w
\end{equation}$$

### Наша цель - оценить все $r^+(y)$ и $r^-(y)$ для $y\in X$, а также найти точки с определенным рангом.





### Опр1: Квантильное разбиение для $D$ определим так -  $Q(D) = (S, \widetilde r^+_{D}, \widetilde r^-_{D}, \widetilde w_{D})$, где
### $S = \{x_1, x_2,\cdots, x_k\}$ выборка точек из $D$~( $x_i \in\{x|(x,w)\in D\}$) со следующими свойствами:

### 1) $x_{i}< x_{i+1} \mbox{ for all } i$, и $x_1$ и $x_k$ - минимум и максимум точек из $D$:
### $$x_1 = \min_{(x,w)\in D} x,\ \  x_k = \max_{(x,w)\in D} x$$
### 2) $r^+_{D}$, $r^-_{D}$ и $w_{D}$  функции в  $S\rightarrow \in [0; \inf)$, удовлетворяющие
###  $$\widetilde r^-_{D}(x_i) \leq  r^-_{D} (x_i),\ \ \widetilde r^+_{D}(x_i) \geq r^+_{D}(x_i), \ \  \widetilde  w(x_i)\leq w_{D}(x_i)$$
### $$ \widetilde r^-_{D}(x_i) + \widetilde w(x_i)\leq \widetilde r^-_{D}(x_{i+1}), \ \ \widetilde r^+_{D}(x_i) \leq \widetilde r^+_{D}(x_{i+1}) - \widetilde w(x_{i+1})$$

### Если у нас есть k элементов, то нам нужно 4k  памяти на этом этапе.


### Опр2. Данное квантильное разбиение $Q(D) = (S, \widetilde r^+_{D}, \widetilde r^-_{D}, \widetilde w_{D})$
### , назовем  $\epsilon$-точным, если $ y \in X$
### $$\begin{equation}\label{eq:epsdef}
\widetilde r^+_{D}(y) - \widetilde r^-_{D}(y) - \widetilde w_{D}(y) \leq  \epsilon w(D)
\end{equation} \ \ \ \ \ \ \ \ \ \ (1)$$
### По определению $r^-(y)\in[\widetilde r^-_{D}(y),\widetilde r^+_{D}(y) -\widetilde w_{D}(y)] $ и $r^+(y)\in[\widetilde r^-_{D}(y)+\widetilde w_{D}(y), \widetilde r^+_{D}(y)] $. Тогда уравнение означает, что мы можем оценить $r^+(y)$ и $r^-(y)$ с ошибкой не более $ \epsilon w(D)$:

### $$(\widetilde r^+_{D}(y) - r^+_{D}(y)) + ( r^-_{D}(y) - \widetilde r^-_{D}(y)) = \widetilde r^+_{D}(y) - \widetilde r^-_{D}(y) - r^+_{D}(y) + r^-_{D}(y)  = \widetilde r^+_{D}(y) - \widetilde r^-_{D}(y) -  w_{D}(y) \ \ \ \  \leq \widetilde r^+_{D}(y) - \widetilde r^-_{D}(y) -  \widetilde w_{D}(y) \leq \epsilon w(D)$$

<img src="pictures/newstrip.png" alt="drawing" width="1400"/>

## Критерий
###  $$Q(D) = (S, \widetilde r^+_{D}, \widetilde r^-_{D}, \widetilde w_{D})$$ $\epsilon$-точное разбиение , тогда и только тогда, когда:

### $$ \widetilde r^+(x_i) - \widetilde r^-(x_i) - \widetilde w(x_i) \leq  \epsilon w(D) \ \ \ \ (2)$$


### $$ \widetilde r^+(x_{i+1}) - \widetilde r^-(x_i) - \widetilde w(x_{i+1}) - \widetilde w(x_{i}) \leq  \epsilon w(D) \ \ \ \ \ (3) $$
### Доказательство: возьмем  $y\in(x_i, x_{i+1})$
### По построению расширения функции , усли $y \in (x_i, x_{i+1})$ для какого-то $i$:
### $$\begin{equation}
\begin{split}
    \widetilde r^-(y)& =\widetilde r^-(x_i)+\widetilde w(x_i), \\
      \widetilde r^+(y)& = \widetilde r^+(x_{i+1}) - \widetilde w(x_{i+1}),\\   \widetilde w(y)& = 0
\end{split}
\end{equation}$$
### $$
\widetilde r^+(y) - \widetilde r^-(y) - \widetilde w(y) =  [\widetilde r^+(x_{i+1})-\widetilde w(x_{i+1})]  - [ \widetilde r^+(x_{i})+\widetilde w(x_{i})] - 0 \leq \epsilon w(D) \ \ \ \ \ \ (при \ \  3)
$$
### То есть 2 и 3 дают нам утверждение 1 

## I Инициализация:
### Возьмем маленькое множество $D = \{(x_1,w_1), (x_2,w_2),\cdots, (x_n,w_n)\}$, построим $Q(D) = (S, \widetilde r^+_{D}, \widetilde r^-_{D}, \widetilde w_{D})$, с $S$ таким что ($S = \{x|(x,w)\in D\}$), and $\widetilde r^+_{D}, \widetilde r^-_{D}, \widetilde w_{D}$ определены:
### $
\widetilde r^+_{D}(x)  = r^+_{D}(x), \ \  \widetilde r^-_{D}(x)  = r^-_{D}(x), \ \ \widetilde w_{D}(x) = w_{D}(x) \mbox{ for } x \in S
$

## II Слияние:
### Если $Q(D_1)$ это $\epsilon_1$-точное разбиение и  $Q(D_2)$  $\epsilon_2$-точное разбиение. Тогда их объединение  $Q(D)$ is $\max(\epsilon_1,\epsilon_2)$-точное разбиеине.
### Для всех$y\in X$, справедливо
### $$
\widetilde r^+_{D}(y) - \widetilde r^-_{D}(y) -\widetilde w_{D}(y)
= [\widetilde r^+_{D_{1}}(y) +\widetilde r^+_{D_{2}}(y)] - [\widetilde r^+_{D_1}(y)+\widetilde r^+_{D_2}(y)] -[\widetilde r^-_{D_1}(y) + \widetilde r^-_{D_2}(y)] \ \ \ \ \ \ \ \ \ \  \ \ \ \\  \ \ \ \ \ \ \ \
\leq \epsilon_1 w(D_1)+\epsilon_2 w(D_2) \leq \max(\epsilon_1,\epsilon_2) w(D_1\cup D_2)
$$

## III Сжатие:
### Лемма: Пусть дано  $\epsilon$-точное разбиение $Q(D) = (S, \widetilde r^+_{D}, \widetilde r^-_{D}, \widetilde w_{D})$ , $x^*=g(Q,d)$ где $g(Q,d)$ - возвращает самый близкий по рангу к d элемент, такой что:
### $$
    d \geq \widetilde r^+_{D}(x^*) - \widetilde w_{D}(x^*) - \frac{\epsilon}{2} w(D)\\
    d \leq \widetilde r^-_{D}(x^*) +\widetilde w_{D}(x^*) + \frac{\epsilon}{2} w(D)
$$


### Теперь мы готовы предоставить функцию сжатия  $Q(D) = (S, \widetilde r^+_{D}, \widetilde r^-_{D}, \widetilde w_{D})$ с $S=\{x_1,x_2,\cdots, x_k\}$ с ограничением по памяти $b$.  Сжатие создает новое разбиение $Q'(D) = (S', \widetilde r^+_{D}, \widetilde r^-_{D}, \widetilde w_{D})$ где $S' =\{x'_1,x'_2, \cdots, x'_{b+1}\}$, и $x'_{i}$ :
### $$ x'_{i} = g\left(Q, \frac{i-1}{b} w(D)\right).$$


<img src="pictures/newstrip.png" alt="drawing" width="1400"/>

## Основная теорема:
### Пусть $Q'(D)$  будет $\epsilon$-точным квантильным разбиением $Q(D)$ с ограниченой памятью $b$ . Тогда $Q'(D)$ будет $(\epsilon+\frac{1}{b})$- точное разбиние.
### Доказательство:
### Воспользуемся критерием (см выше) и леммой, докажем, что:
### $$ \frac{i-1}{b}w(D)   + \frac{\epsilon}{2} w(D)  \geq \widetilde r^+_{D}(x'_i) - \widetilde w_{D}(x'_i)\\
    \frac{i-1}{b}w(D)   - \frac{\epsilon}{2} w(D) \leq \widetilde r^-_{D}(x'_i) + \widetilde w_{D}(x'_i) $$ 
### Откуда
### $$ \widetilde r^+_{D}(x'_{i+1}) - \widetilde w_{D}(x'_{i+1}) \leq \frac{i}{b}w(D)   + \frac{\epsilon}{2} w(D) \\
   - \widetilde r^-_{D}(x'_i) - \widetilde w_{D}(x'_i) \leq - \frac{i-1}{b}w(D)   + \frac{\epsilon}{2} w(D) \ $$
 
###  $$\widetilde r^+_{D}(x'_{i+1}) - \widetilde w_{D}(x'_{i+1})  -  \widetilde r^-_{D}(x'_{i}) - \widetilde w_{D} )x'_{i}) \leq \\
 \leq [ \frac{i}{b}w(D) + \frac{\epsilon}{2} w(D)] - [ \frac{i-1}{b}w(D) - \frac{\epsilon}{2} w(D)] = (\frac{1}{b} + \epsilon)w(D)\ \ \ (4)$$ 


<img src="pictures/newstrip.png" alt="drawing" width="1400"/>

## <font size=5 color=black> <center> 3.5 Разбиение для разреженных данных </center> </font> 

<img src="pictures/tree_default.png" alt="drawing" width="700"/>

## Опишем алгоритм поиска дефолтного листа:

### $I_k = \{i \in I | x_{ik} \not = NaN\}$ , $I$ - набор данных в этом узле.
### $G \leftarrow \sum_{i\in I},  g_{i}$
### $H \leftarrow \sum_{i \in I} h_{i}$
### Для всех признаков создадим цикл и будем на каждом шаге проверят,  влево или вправо направить объекты с пустым признаком:
### $G_L\leftarrow 0,\ H_L\leftarrow 0$
### $For\{\ j \  in \ sorted(I_k, ascent  \ order \ by  \ x_{jk})\}$
### $G_L\leftarrow G_L + g_j,\ H_L\leftarrow H_L + h_j$
### $G_R\leftarrow G - G_L,\ H_R\leftarrow H - H_L$
### $score \leftarrow \max(score, \frac{G_{L}^2}{H_{L}+\lambda} + \frac{G_{R}^2}{H_{R}+\lambda} - \frac{G^2}{H+\lambda})$
### Затем аналогично для левого листа

<img src="pictures/allstate-sparsity-1.png" alt="drawing" width="700"/>

<img src="pictures/newstrip.png" alt="drawing" width="1400"/>

##  <font size=5 color=black> <center> 4. Дизайн системы  </center></font> 

<img src="pictures/core.jpg" alt="drawing" width="400" /> 

##  <font size=5 color=black> <center> 4.1 Блочность данных для параллельных вычислений  </center></font> 

## Асимптотика
### d - максимальная глубина дерева
### K - число деревьев
### n - количество объектов 
### $\|x\|_0$ - количество непустых значений во входе
### 1. Авторы утверждают, что для точного жадного алгоритма асимптотика $O(K d \|x\|_0 \log n)$. Проверим: 
### 1.1 Обход K деревьев и по каждому этажу - отсюда $O(K d)$
### 1.2 В каждом узле нам нужно определить оптимальное разбиение, для этого нужно отсортировать все объекты по признаку. Пусть у нас входная матрица $ n \times m$ - n обьектов и у каждого m признаков и пусть $|x_{0i}|$ - количество не пустых значений для каждого признака. Тогда:
### $$\sum_{i=1}^m |x_{0i}| \cdot \log |x_{0i}| \leq \sum_{i=1}^m |x_{0i}| \cdot \log n = \|x\|_0 \cdot \log n$$ тогда асимптотика  $O(K d \|x\|_0 \log n)$.





### 2. Для блочной структуры: $O(K d \|x\|_0 + \|x\|_0 \log n)$, где $O( \|x\|_0 \log n)$ - время сортировки и может быть еще меньше при определенных условиях. $O(K d \|x\|_0)$ - так как данные уже отсортированны изначально.
<img src="pictures/data_layout.png" alt="drawing" width="1000" /> 


<img src="pictures/newstrip.png" alt="drawing" width="1400"/>

##  <font size=5 color=black> <center> 4.2 Работа с кэшем </center></font> 


In [3]:
from IPython.display import HTML, display
display(HTML("<table><tr><td><img src='pictures/allstate-cache-bynthread-10m-1.png'></td><td><img src='pictures/allstate-cache-bynthread-1m-1.png'></td></tr></table>"))


,


  # <font size=4 color=black> <center>      10M и 1M  объектов </center></font> 

<img src="pictures/allstate-approx-byblock-10m-1.png" alt="drawing" width="500" /> 

<img src="pictures/table.png" alt="drawing" width="1000" /> 

<img src="pictures/newstrip.png" alt="drawing" width="1400"/>